In [1]:
#Проверка доступносит cuda
import torch
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [ ]:
#для работы необходиv модуль fast ai, который облегчает читаемость кода, при этом 
! pip install fastai

In [0]:
#импортируем нужные модули машинного обучения
import random
import numpy as np
import pandas as pd
import os
#импортируем модуль Fast AI
from fastai import *
#модуль для работы с графикой
from fastai.vision import *
#модуль callbacks для интеграциии дополняющего модуля сохранения лучше модели
from fastai.callbacks import SaveModelCallback

In [0]:
#обновление рандома для обеспечения повторяемости
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

In [ ]:
#открываем архив с google drive и распаковываем на colab
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
!unzip -q /content/gdrive/My\ Drive/Simpsons/data/simpsons4.zip

In [0]:
#указываем путь к папке train
path = 'train/'
#batch size
bs = 84
seed_everything()
#ImageDataBunch.from_folder загружает и нормализует изображения из папки train, 
#аугментирует их при помощи всех трансформаций из get_transforms()
#доля валидационной выборки 0.2, размер картинок 224 пикселя 
data = ImageDataBunch.from_folder(path, ds_tfms=get_transforms(), valid_pct=0.2, size=224, num_workers=4, bs=bs).normalize()

In [6]:
#задаём модель ResNet34, отслеживаемая метрика accuracy, оптимизатор AdamW
learn = cnn_learner(data, models.resnet34, metrics=accuracy, opt_func = optim.AdamW)
#размораживаем веса из готовой модели
learn.unfreeze()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 48.2MB/s]


In [0]:
seed_everything()
#обучаем модель согласно one cycle policy, что увеличивает скорость и точность обучения
#SaveModelCallback сохраняет лучшую модель на основе метрики accuracy
learn.fit_one_cycle(15, callbacks=[SaveModelCallback(learn, every='improvement', monitor='accuracy', name='best')])
learn.load('best') #загрузка лучшеё модели
learn.export() #экспортируем лучшую версию модели

epoch,train_loss,valid_loss,accuracy,time


In [0]:
test = ImageList.from_folder('testset/testset') #загружаем тестовые картинки
learn = load_learner('train', test=test) #загружаем сохранённую лучшую модель
preds, _ = learn.get_preds(ds_type=DatasetType.Test) # получаем предсказания

#список самых вероятных меток из предсказанных:
labelled_preds = [' '.join([learn.data.classes[i] for i,p in enumerate(pred) if p == max(pred)]) for pred in preds] 
fnames = [f.name for f in learn.data.test_ds.items] #список имён файлов
df = pd.DataFrame({'Id':fnames, 'Expected':labelled_preds}, columns=['Id', 'Expected']) #объединяем всё в датафрэйм

In [0]:
len(preds) #проверяем, все ли предсказания получены

991

In [0]:
#сортировка значений в датафрэйме, необходимость появляется именно при выполнении на colab
df = df.sort_values('Id', ascending=True)

In [0]:
df.head() #проверка правильности отображения данных

,Id,Expected
185,img0.jpg,nelson_muntz
809,img1.jpg,bart_simpson
273,img10.jpg,ned_flanders
638,img100.jpg,chief_wiggum
674,img101.jpg,apu_nahasapeemapetilon


In [0]:
df.to_csv('submission.csv', index=False) #экспортируем предсказания в csv